# Decorator Basics
# Python’s functions are objects
To understand decorators, you must first understand that functions are objects in Python. This has important consequences. Let’s see why with a simple example :

In [53]:
def greet(word='Hello'):
    return word.capitalize() + '!'

print(greet())


scream = greet

# Notice we don’t use parentheses: we are not calling the function, we are
# putting the function `shout` into the variable `scream`. 
# It means you can then call `greet` from `scream`:

print(scream())


del greet
try:
    print(greet())
except NameError as e:
    print(e)


print(scream())

Hello!
Hello!
name 'greet' is not defined
Hello!


Python functions is they can be defined... inside another function!

In [7]:
def talk():


    def whisper(word='yes'):
        return word.lower() + '...'

    print (whisper())

talk()


try:
    print (whisper())
except NameError as e:
    print (e)


yes...
name 'whisper' is not defined


 function can return another function.

In [55]:
def getTalk(kind='greet'):

    # We define functions on the fly
    def greet(word='greet'):
        return word.capitalize() + '!'

    def whisper(word='yes'):
        return word.lower() + '...'

    # Then we return one of them
    if kind == 'greet':
        return greet  
    else:
        return whisper

talk = getTalk()      

print(talk)

print (talk())

print (getTalk('whisper')())


<function getTalk.<locals>.greet at 0x7fe590edba60>
Greet!
yes...


In [11]:
def doSomethingBefore(func): 
    print ('I do something before then I call the function you gave me')
    print (func())

doSomethingBefore(scream)

I do something before then I call the function you gave me
Yes!


Well, you just have everything needed to understand decorators. You see, decorators are “wrappers”, which means that they let you execute code before and after the function they decorate without modifying the function itself.

# Handcrafted decorators

In [13]:
# A decorator is a function that expects ANOTHER function as parameter
def my_shiny_new_decorator(a_function_to_decorate):

    def the_wrapper_around_the_original_function():

        # Put here the code you want to be executed BEFORE the original 
        # function is called
        print ('Before the function runs')

        # Call the function here (using parentheses)
        a_function_to_decorate()

        # Put here the code you want to be executed AFTER the original 
        # function is called
        print ('After the function runs')

    return the_wrapper_around_the_original_function

def a_stand_alone_function():
    print ('I am a stand alone function, don’t you dare modify me')

a_stand_alone_function() 


a_stand_alone_function_decorated = my_shiny_new_decorator(a_stand_alone_function)
a_stand_alone_function_decorated()

I am a stand alone function, don’t you dare modify me
Before the function runs
I am a stand alone function, don’t you dare modify me
After the function runs


In [14]:
a_stand_alone_function = my_shiny_new_decorator(a_stand_alone_function)
a_stand_alone_function()


Before the function runs
I am a stand alone function, don’t you dare modify me
After the function runs


# using the decorator syntax:

In [16]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print ('Leave me alone')

another_stand_alone_function()  

Before the function runs
Leave me alone
After the function runs


#Yes, that’s all, it’s that simple. @decorator is just a shortcut to:


another_stand_alone_function = my_shiny_new_decorator(another_stand_alone_function)

In [47]:
def bread(func):
    def wrapper():
        print( "Hello Here is the reciepe")
        func()
        print ("Bye BYe")
    return wrapper

def ingredients(func):
    def wrapper():
        print ('#tomatoes#')
        func()
        print ('~salad~')
    return wrapper

def sandwich(food='--ham--'):
    print (food)

sandwich()

sandwich = bread(ingredients(sandwich))
sandwich()

--ham--
Hello Here is the reciepe
#tomatoes#
--ham--
~salad~
Bye BYe


Using the Python decorator syntax:

In [48]:
@bread
@ingredients
def sandwich(food='--ham--'):
    print(food)
sandwich()

Hello Here is the reciepe
#tomatoes#
--ham--
~salad~
Bye BYe


The order you set the decorators MATTERS:

In [49]:
@ingredients
@bread
def strange_sandwich(food='--ham--'):
    print(food)

strange_sandwich()

#tomatoes#
Hello Here is the reciepe
--ham--
Bye BYe
~salad~


# Passing arguments to the decorated function

In [29]:
# It’s not black magic, you just have to let the wrapper 
# pass the argument:

def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):
        print('I got args! Look:', arg1, arg2)
        function_to_decorate(arg1, arg2)
    return a_wrapper_accepting_arguments

# Since when you are calling the function returned by the decorator, you are
# calling the wrapper, passing arguments to the wrapper will let it pass them to 
# the decorated function

@a_decorator_passing_arguments
def print_full_name(first_name, last_name):
    print('My name is', first_name, last_name)
    
print_full_name('Peter', 'Venkman')

I got args! Look: Peter Venkman
My name is Peter Venkman


# Passing arguments to the decorator
Great, now what would you say about passing arguments to the decorator itself?

This can get somewhat twisted, since a decorator must accept a function as an argument. Therefore, you cannot pass the decorated function’s arguments directly to the decorator.


In [37]:
# Decorators are ORDINARY functions
def my_decorator(func):
    print('I am an ordinary function')
    def wrapper():
        print('I am function returned by the decorator')
        func()
    return wrapper

# Therefore, you can call it without any '@'

def lazy_function():
    print('zzzzzzzz')

decorated_function = my_decorator(lazy_function)


@my_decorator
def lazy_function():
    print ('zzzzzzzz')


I am an ordinary function
I am an ordinary function


In [42]:
def decorator_with_args(decorator_to_enhance):
    """ 
    This function is supposed to be used as a decorator.
    It must decorate an other function, that is intended to be used as a decorator.
    Take a cup of coffee.
    It will allow any decorator to accept an arbitrary number of arguments,
    saving you the headache to remember how to do that every time.
    """
    
    # We use the same trick we did to pass arguments
    def decorator_maker(*args, **kwargs):
       
        # We create on the fly a decorator that accepts only a function
        # but keeps the passed arguments from the maker.
        def decorator_wrapper(func):
    
            return decorator_to_enhance(func, *args, **kwargs)
        
        return decorator_wrapper
    
    return decorator_maker

In [43]:
# Don’t forget, the signature is `decorator(func, *args, **kwargs)`
@decorator_with_args 
def decorated_decorator(func, *args, **kwargs): 
    def wrapper(function_arg1, function_arg2):
        print ('Decorated with', args, kwargs)
        return func(function_arg1, function_arg2)
    return wrapper
    
# Then you decorate the functions you wish with your brand new decorated decorator.

@decorated_decorator(42, 404, 1024)
def decorated_function(function_arg1, function_arg2):
    print ('Hello', function_arg1, function_arg2)

decorated_function('Universe and', 'everything')


Decorated with (42, 404, 1024) {}
Hello Universe and everything


Some other simple example


In [52]:
NAMES = set()


def replace(old, new):
    """Replace the first function argument.
    If the first function argument matches old it will be Replaced with new.
    """
    # decorate the original function
    def decorate(func):
        # do the replacement using the args of the decorator
        def do_replace(*args, **kwargs):
            if args[0] == old:
                args = (new,)
            # call the decorated function
            return func(*args, **kwargs)
        return do_replace
    return decorate



def remember(func):
    """Remember each name that was called.
    Does only remember each name once.
    """
    # wrap the original function to fetch it's arguments
    def wrapper(*args, **kwargs):
        NAMES.add(args[0])
        # call the decorated function
        return func(*args, **kwargs)
    return wrapper


def check_allowed(func):
    """Check if a function is allowed.
    Returns a function which raises a NameError if a function is not in the
    allowed list.
    """
    allowed = ['say_hi']
    if func.__name__ not in allowed:
        def not_allowed(*args):
            raise NameError('%s not allowed' % func.__name__)
        return not_allowed
    return func


@replace('Ruchi', 'Bhardwaj')
@remember
@check_allowed
def say_hi(name):
    return "Hi %s" % name


def say_hello(name):
    return "Hello %s" % name
say_hello = remember(check_allowed(say_hello))


QUEUE = [
    (say_hi, 'Ruchi'),
    (say_hello, 'Ruchi'),
    (say_hi, 'Ruchi'),
    (say_hello, 'Ruchi'),
    (say_hi, 'Ruchi'),
    (say_hi, 'Ruchi'),
]
for func, name in QUEUE:
    try:
        print( 'Will greet %s' % name)
        print (func(name))
    except NameError:
        print ('%s is not allowed' % name)
print ('names used: %s' % ', '.join(NAMES))

Will greet Ruchi
Hi Bhardwaj
Will greet Ruchi
Ruchi is not allowed
Will greet Ruchi
Hi Bhardwaj
Will greet Ruchi
Ruchi is not allowed
Will greet Ruchi
Hi Bhardwaj
Will greet Ruchi
Hi Bhardwaj
names used: Ruchi, Bhardwaj


# Descriptors


In [56]:
class Descriptor(object):
    def __init__(self,name=None):
        self._name = name

    def __get__(self, instance, owner):
        print("__get__")
        return instance

    def __set__(self, instance, value):
        print("__set__")
        instance.__dict__[self._name] = value

    def __delete__(self, instance):
        print("__delete__")
        del instance.__dict__[self._name]

    def __repr__(self):
        return "Object Descriptor :{}".format(self._name)

class People(Descriptor):
    name = Descriptor()

    def MethoA(self):
        print("===", self.name)


if __name__=="__main__":
    obj = People(name="Ruchi")
    print(obj)
    print(obj.name)
    print(obj.MethoA())


Object Descriptor :Ruchi
__get__
Object Descriptor :Ruchi
__get__
=== Object Descriptor :Ruchi
None
